In [0]:
import pandas as pd
import geopandas
import itertools
import pickle
from sklearn import preprocessing
from sklearn import kernel_ridge
import numpy as np

In [0]:
!pip install geopandas

     |████████████████████████████████| 901kB 38.9MB/s 
     |████████████████████████████████| 11.2MB 47.7MB/s 
     |████████████████████████████████| 17.9MB 44.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch


In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.9 GB  | Proc size: 228.8 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


In [0]:
#Try to browse drive files
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#import point data
fw_pts = geopandas.read_file('drive/My Drive/10k_Points_S2_VI_S1_BP.shp')
#points = fw_pts
fw_pts.head()

,B2,B3,B4,B5,B6,B7,B8,B8A,B11,B12,arvi,dvi,gemi,gndvi,ipvi,ireci,mcari,msavi,msavi2,mtci,ndi45,pssra,pvi,reip,rvi,s2rep,savi,tndvi,tsavi,wdvi,S0_VH_N,S0_VV_N,lai,lai_cab,lai_cw,fapar,fcover,geometry
0,1017.0,1355.0,1884.0,2150.0,2365.0,2598.0,2718.0,3009.0,4248.0,3156.0,-0.006034,0.0834,0.474812,0.314445,0.590613,0.078540,0.012211,0.114926,0.616914,0.808270,0.065940,1.378981,0.058973,716.930237,1.442675,719.813965,0.130285,0.825364,-0.924842,0.1776,0.014561,0.053904,0.293115,9.517295,0.005263,0.031921,0.018619,POINT (340065 395145)
1,1057.0,1372.0,1896.0,2150.0,2365.0,2598.0,2770.0,3009.0,4248.0,3156.0,0.006358,0.0874,0.472472,0.308816,0.593656,0.077220,0.011158,0.119945,0.622073,0.846456,0.062778,1.370253,0.061801,718.046509,1.460971,720.790710,0.135630,0.829043,-0.892195,0.1822,0.014947,0.052304,0.292749,9.420005,0.005250,0.030687,0.017791,POINT (340075 395145)
2,1072.0,1397.0,1914.0,2203.0,2415.0,2681.0,2776.0,3108.0,4254.0,3111.0,0.003615,0.0862,0.482500,0.314860,0.591898,0.084081,0.014710,0.118084,0.620134,0.733564,0.070197,1.400731,0.060953,717.830200,1.450366,720.601440,0.133437,0.826919,-0.882630,0.1819,0.014947,0.052304,0.306748,9.406707,0.005524,0.036503,0.023166,POINT (340085 395145)
3,1072.0,1421.0,1936.0,2203.0,2415.0,2681.0,2791.0,3108.0,4254.0,3111.0,-0.001610,0.0855,0.478173,0.307167,0.590438,0.081669,0.012585,0.116814,0.618799,0.794007,0.064508,1.384814,0.060458,719.905640,1.441632,722.417419,0.131849,0.825152,-0.867322,0.1823,0.016830,0.051905,0.306178,9.276816,0.005510,0.034058,0.021415,POINT (340095 395145)
4,1123.0,1459.0,2048.0,2264.0,2585.0,2856.0,2957.0,3258.0,4284.0,3098.0,-0.002698,0.0909,0.475939,0.323754,0.590809,0.092256,0.006080,0.121885,0.623883,1.486111,0.050093,1.394531,0.064276,723.426819,1.443848,725.498413,0.136282,0.825602,-0.756723,0.1933,0.016830,0.051905,0.330857,9.463584,0.005839,0.048685,0.030191,POINT (340105 395145)


In [0]:
#S2 Ratios
bands = ['B2.', 'B3.', 'B4.', 'B5.', 'B6.', 'B7.', 'B8.', 'B8A.', 'B11.', 'B12.']
combos = list(map(''.join, itertools.chain(itertools.product(bands, bands))))
numerator = [None] * 100
denominator = [None] * 100

for i, string in enumerate(combos):
  split = string.split(".")
  if split[0] == split[1]:
    next
  else:
    numerator[i] = split[0]
    denominator[i] = split[1]
    
numerator = [x for x in numerator if x is not None]
denominator = [x for x in denominator if x is not None]

In [0]:
#make list of new column names
rat_names = [None] * len(numerator)
for i, band in enumerate(numerator):
  name = str(numerator[i]) + 'v' + str(denominator[i])
  rat_names[i] = name

In [0]:
#Perform the ratio division, put that data into a new column with proper name
for i, col_name in enumerate(rat_names):
  fw_pts.loc[:, col_name] = fw_pts.loc[:, numerator[i]] / fw_pts.loc[:, denominator[i]]

In [0]:
rat_names = ['VVvVH', 'VHvVV']
fw_pts.loc[:, rat_names[0]] = fw_pts.loc[:, 'S0_VV_N'] / fw_pts.loc[:, 'S0_VH_N']
fw_pts.loc[:, rat_names[1]] = fw_pts.loc[:, 'S0_VH_N'] / fw_pts.loc[:, 'S0_VV_N']

In [0]:
fw_pts.to_file(driver = 'ESRI Shapefile', filename= "drive/My Drive/10k_Points_S2_VI_S1_BP_S2RAT_S1RAT.shp")

KeyboardInterrupt: ignored

In [0]:
#Check the number of columns
len(list(fw_pts))

130

In [0]:
#Names of the independent variables that will be predicted:
y_col_names = ['Dry_wgh', 'Frsh_wg', 'CP']
y_col_full_names = ['Dry Weight Biomass', 'Fresh Weight Biomass', 'Crude Protein']

#Names of the predictor variables
X_col_names = list(fw_pts.loc[:, fw_pts.columns != 'geometry'])

In [0]:
print(y_col_names)

['Dry_wgh', 'Frsh_wg', 'CP']


In [0]:
print(X_col_names)

['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12', 'arvi', 'dvi', 'gemi', 'gndvi', 'ipvi', 'ireci', 'mcari', 'msavi', 'msavi2', 'mtci', 'ndi45', 'pssra', 'pvi', 'reip', 'rvi', 's2rep', 'savi', 'tndvi', 'tsavi', 'wdvi', 'S0_VH_N', 'S0_VV_N', 'lai', 'lai_cab', 'lai_cw', 'fapar', 'fcover', 'B2vB3', 'B2vB4', 'B2vB5', 'B2vB6', 'B2vB7', 'B2vB8', 'B2vB8A', 'B2vB11', 'B2vB12', 'B3vB2', 'B3vB4', 'B3vB5', 'B3vB6', 'B3vB7', 'B3vB8', 'B3vB8A', 'B3vB11', 'B3vB12', 'B4vB2', 'B4vB3', 'B4vB5', 'B4vB6', 'B4vB7', 'B4vB8', 'B4vB8A', 'B4vB11', 'B4vB12', 'B5vB2', 'B5vB3', 'B5vB4', 'B5vB6', 'B5vB7', 'B5vB8', 'B5vB8A', 'B5vB11', 'B5vB12', 'B6vB2', 'B6vB3', 'B6vB4', 'B6vB5', 'B6vB7', 'B6vB8', 'B6vB8A', 'B6vB11', 'B6vB12', 'B7vB2', 'B7vB3', 'B7vB4', 'B7vB5', 'B7vB6', 'B7vB8', 'B7vB8A', 'B7vB11', 'B7vB12', 'B8vB2', 'B8vB3', 'B8vB4', 'B8vB5', 'B8vB6', 'B8vB7', 'B8vB8A', 'B8vB11', 'B8vB12', 'B8AvB2', 'B8AvB3', 'B8AvB4', 'B8AvB5', 'B8AvB6', 'B8AvB7', 'B8AvB8', 'B8AvB11', 'B8AvB12', 'B11vB2', 'B11vB3',

In [0]:
#Define predictor variables
X = fw_pts.loc[:, X_col_names]

In [0]:
#import selected features
optimize_names = pd.read_csv('drive/My Drive/Optimized_Variable_Names.csv')

In [0]:
#PREDICTIVE SECTION

In [0]:
#Predict Dry_wgh varible
#select the optimized features and assign to X
var = ['Dry_wgh']
X_opt = X.loc[:, list(optimize_names.loc[optimize_names[var[0]].notna(), var[0]])]

#Something is causing all these na values
X_opt = X_opt.dropna(axis=0, how='any')
X_opt2 = X_opt.dropna(axis=0, how='any')
print('Any Nan? ' + str(X_opt.isnull().values.any()))
print(len(X_opt.iloc[:, 0]))

Any Nan? False
557317


In [0]:
for col in list(X_opt):
  X_opt.loc[:, col] = preprocessing.scale(np.array(X_opt.loc[:, col]))
X_opt = X_opt.to_numpy()

In [1]:
#predict values for each point
reg = pickle.load(open('drive/My Drive/Saved_Models/Dry_wgh_model.p', 'rb' ) )

NameError: ignored

In [0]:
pred = reg.predict(X_opt)

In [0]:
#unscale predictions
scalery = pickle.load(open('drive/My Drive/Saved_Models/Dry_wgh_scaler.p', 'rb' ) )
pred_unscaled = scalery.inverse_transform(pred)

In [0]:
#reattach predictions to points
#type(fw_pts.loc[:, 'geometry'])
point_preds = geopandas.GeoDataFrame(fw_pts, geometry='geometry')
#point_preds = point_preds['geometry']
type(point_preds)

geopandas.geodataframe.GeoDataFrame

In [0]:
print(len(X_opt2.index), len(pred_unscaled))

557317 557317


In [0]:
cn = var[0] + '_Pred'
point_preds.loc[X_opt2.index, cn] = pred_unscaled

In [0]:
#Repeat for Frsh_wg
var = ['Frsh_wg']
X_opt = X.loc[:, list(optimize_names.loc[optimize_names[var[0]].notna(), var[0]])]
X_opt = X_opt.dropna(axis=0, how='any')
X_opt2 = X_opt.dropna(axis=0, how='any')
print('Any Nan? ' + str(X_opt.isnull().values.any()))
print(len(X_opt.iloc[:, 0]))

Any Nan? False
557317


In [0]:
for col in list(X_opt):
  X_opt.loc[:, col] = preprocessing.scale(np.array(X_opt.loc[:, col]))
X_opt = X_opt.to_numpy()

In [0]:
path = 'drive/My Drive/Saved_Models/'
model = path + var[0] + '_model.p'
reg = pickle.load(open(model, 'rb' ) )
pred = reg.predict(X_opt)

scaler = path + var[0] + '_scaler.p'
scalery = pickle.load(open(scaler, 'rb' ) )
pred_unscaled = scalery.inverse_transform(pred)

In [0]:
cn = var[0] + '_Pred'
point_preds.loc[X_opt2.index, cn] = pred_unscaled

In [0]:
#Repeat for CP
var = ['CP']
X_opt = X.loc[:, list(optimize_names.loc[optimize_names[var[0]].notna(), var[0]])]
X_opt = X_opt.dropna(axis=0, how='any')
X_opt2 = X_opt.dropna(axis=0, how='any')
print('Any Nan? ' + str(X_opt.isnull().values.any()))
print(len(X_opt.iloc[:, 0]))


Any Nan? False
558006


In [0]:
for col in list(X_opt):
  X_opt.loc[:, col] = preprocessing.scale(np.array(X_opt.loc[:, col]))
X_opt = X_opt.to_numpy()


In [0]:
path = 'drive/My Drive/Saved_Models/'
model = path + var[0] + '_model.p'
reg = pickle.load(open(model, 'rb' ) )
pred = reg.predict(X_opt)

scaler = path + var[0] + '_scaler.p'
scalery = pickle.load(open(scaler, 'rb' ) )
pred_unscaled = scalery.inverse_transform(pred)

In [0]:
cn = var[0] + '_Pred'
point_preds.loc[X_opt2.index, cn] = pred_unscaled

In [0]:
point_preds = point_preds.dropna(axis=0, how='any')

In [0]:
point_preds = point_preds.loc[:, ['geometry', 'Dry_wgh_Pred', 'Frsh_wg_Pred', 'CP_Pred']]

In [0]:
point_preds

,geometry,Dry_wgh_Pred,Frsh_wg_Pred,CP_Pred
0,POINT (340065 395145),0.020078,0.270731,9.587198
1,POINT (340075 395145),0.018084,0.264475,9.875035
2,POINT (340085 395145),0.026635,0.314345,9.749200
3,POINT (340095 395145),0.010157,0.281725,10.417368
4,POINT (340105 395145),0.022244,0.322885,10.554080
5,POINT (340115 395145),0.014614,0.292191,10.740750
6,POINT (340125 395145),0.032385,0.308875,10.669192
7,POINT (340135 395145),0.040233,0.291514,10.409178
8,POINT (340145 395145),0.053547,0.305231,9.922849
9,POINT (340155 395145),0.052022,0.265799,10.050304


In [0]:
point_preds.to_file(driver = 'ESRI Shapefile', filename= "drive/My Drive/Point_Predictions.shp")